In [6]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [22]:
env = GridworldEnv()

In [66]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy. S: 0, 1, 2, ..., 15. A: 0(UP), 1(RIGHT), 2(DOWN), 3(LEFT)
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    # Initiallize the value function for the next iteration update
    V_next = np.zeros(env.nS)
    
    # value function after one iteration update
    for s in range(env.nS):
        for a in range(env.nA):
            (prob, next_state, reward, done) = env.P[s][a][0]
            V_next[s] += policy[s][a]*(reward + discount_factor * V[next_state])
            
    changes = [abs(i - j) for i, j in zip(V_next,V)]
    
    # stop policy evaluation once the value function change is less then theta for all states
    while any([change > theta for change in changes]):
        V = V_next
        V_next = np.zeros(env.nS)
        
        for s in range(env.nS):
            for a in range(env.nA):
                prob, next_state, reward, done = env.P[s][a][0]
                V_next[s] += policy[s][a]*(reward + discount_factor * V[next_state])
                
        changes = [abs(i - j) for i, j in zip(V_next,V)]       

    return np.array(V_next)

In [67]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [68]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

print(v)

[  0.         -13.99989315 -19.99984167 -21.99982282 -13.99989315
 -17.99986052 -19.99984273 -19.99984167 -19.99984167 -19.99984273
 -17.99986052 -13.99989315 -21.99982282 -19.99984167 -13.99989315   0.        ]


In [69]:
# misc testing 

a = [1, 2, 3, 4]
b = [1.01, 2.01, 3.01, 4.01]
diff = [abs(i - j) for i, j in zip(a,b)]

print (any([d < 0 for d in diff]))

False
